### Example of DNNs in regression - Housing dataset

Loading data

In [2]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape)
print(test_data.shape)
print(max(train_targets), min(train_targets))

(404, 13)
(102, 13)
50.0 5.0


Standardizing data

In [3]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

Building the model (deine function to enable more flexibility in tests)

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model(hidden = 64):
    model = models.Sequential()
    model.add(layers.Dense(hidden, activation='relu',
                            input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(hidden, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [5]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=1)

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mse_score, test_mae_score)

Epoch 1/80


2023-02-27 14:58:51.641487: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-27 14:58:51.641524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (franciscoizquierdo-GF65-Thin-10UE): /proc/driver/nvidia/version does not exist
2023-02-27 14:58:51.641978: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


26/26 [==============================] - 0s 857us/step - loss: 497.3827 - mae: 20.4491
Epoch 2/80
26/26 [==============================] - 0s 738us/step - loss: 335.2344 - mae: 16.2579
Epoch 3/80
26/26 [==============================] - 0s 715us/step - loss: 167.3736 - mae: 10.6246
Epoch 4/80
26/26 [==============================] - 0s 689us/step - loss: 74.9106 - mae: 6.4954
Epoch 5/80
26/26 [==============================] - 0s 783us/step - loss: 43.4666 - mae: 4.7962
Epoch 6/80
26/26 [==============================] - 0s 745us/step - loss: 31.2724 - mae: 3.9965
Epoch 7/80
26/26 [==============================] - 0s 656us/step - loss: 25.9693 - mae: 3.6228
Epoch 8/80
26/26 [==============================] - 0s 713us/step - loss: 23.3551 - mae: 3.4303
Epoch 9/80
26/26 [==============================] - 0s 702us/step - loss: 20.9893 - mae: 3.1901
Epoch 10/80
26/26 [==============================] - 0s 656us/step - loss: 19.3960 - mae: 3.0709
Epoch 11/80
26/26 [=========================

In [7]:
import numpy as np
k = 5
num_val_samples = len(train_data) // k 
num_epochs = 100
all_scores = []
model = build_model()
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
    partial_train_data = np.concatenate( [train_data[:i*num_val_samples], train_data[(i+1)*num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate( [train_targets[:i*num_val_samples], train_targets[(i+1)*num_val_samples:]], axis=0)
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
print(all_scores)
print(np.mean(all_scores))

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
[1.885538101196289, 2.7977218627929688, 2.282827854156494, 1.7222213745117188, 1.5181291103363037]
2.041287660598755
